# API Tests

In [49]:
import random, string

def gen_url():
    letters = string.ascii_lowercase
    protocols = ["http://", "https://", "ftp://"]
    level1s = [".com", ".ru", ".fr", ".uk"]
    domain = ''.join(random.choice(letters) for i in range(8))
    path = ''.join(random.choice(letters) for i in range(15))
    return random.choice(protocols) + domain + random.choice(level1s) + "/" + path

## 1. GRPC

### 1.1 Set up

In [50]:
!python -m grpc_tools.protoc -I../api/proto/ --python_out=. --pyi_out=. --grpc_python_out=. ../api/proto/links.proto

In [51]:
import grpc
import links_pb2_grpc

channel = grpc.insecure_channel('localhost:50051')
stub = links_pb2_grpc.LinkServiceStub(channel)

In [52]:
import links_pb2
import re

status_pattern = re.compile("StatusCode.([A-Z_]+)\n")

def extract_error(string):
    return status_pattern.search(string).group(1)

def create_link_grpc(url):
    try:
        request = links_pb2.CreateLinkRequest(url=str(url))
        result = stub.CreateLink(request)
        return result.alias
    except grpc.RpcError as e:
        return extract_error(str(e))
        

def get_link_grpc(alias):
    try:
        request = links_pb2.GetLinkRequest(alias=str(alias))
        result = stub.GetLink(request)
        return result.url
    except grpc.RpcError as e:
        return extract_error(str(e))

def grpc_must_already_exists(url):
    return create_link_grpc(url) == "ALREADY_EXISTS"

def grpc_must_not_found(alias):
    return get_link_grpc(url) == "NOT_FOUND"

def grpc_must_bad_input(url):
    return create_link_grpc(url) == "INVALID_ARGUMENT"

def grpc_must_ok():
    url = gen_url()
    alias = create_link_grpc(url)
    resp = get_link_grpc(alias)
    res = resp == url
    if not res:
        print(f"Wrong result for:\nurl: {url}\nalias: {alias}\ngot response: {resp}")
    return res

### 1.2 Run tests

#### 1.2.1 Launch app

In [53]:
!sudo docker compose -f with_flags-compose.yml up -d

[+] Running 2/0
 ✔ Container golinkcut_db_test      Runni...                               0.0s 
 ✔ Container for-tests-golinkcut-1  R...                                   0.0s 


#### 1.2.2 Run tests

In [55]:
print("trying normal tests:")
ok = True
for i in range(15):
    if not grpc_must_ok():
        ok = False
if ok:
    print("OK")

print("tests with errors:")

ok = True
url = gen_url()
alias = create_link_grpc(url)
if grpc_must_already_exists(url):
    print("OK")
else:
    print(f"Could not get ERROR_EXISTS_ALREADY")

if grpc_must_not_found("0123456789"):
    print("OK")
else:
    print("Could not get NOT_FOUND_ERROR")

if grpc_must_bad_input("not-a-url-at-all"):
    print("OK")
else:
    print(f"Wrong! Should be INVALID_ARGUMENT")

trying normal tests:
OK
tests with errors:
OK
OK
OK


#### 1.2.3 Stop app

In [56]:
!cd .. && sudo docker compose -f ./for-tests/with_flags-compose.yml down

[+] Running 1/0
 ✔ Container golinkcut_db_test      Remov...                               0.0s 
 ⠋ Container for-tests-golinkcut-1  S...                                   0.1s 
[+] Running 1/2
 ✔ Container golinkcut_db_test      Remov...                               0.0s 
 ⠙ Container for-tests-golinkcut-1  S...                                   0.2s 
[+] Running 2/2
 ✔ Container golinkcut_db_test      Remov...                               0.0s 
 ✔ Container for-tests-golinkcut-1  R...                                   0.3s 
 ⠋ Network for-tests_default        Removin...                             0.0s 
[+] Running 2/3
 ✔ Container golinkcut_db_test      Remov...                               0.0s 
 ✔ Container for-tests-golinkcut-1  R...                                   0.3s 
 ⠙ Network for-tests_default        Removin...                             0.1s 
[+] Running 3/3
 ✔ Container golinkcut_db_test      Remov...                               0.0s 
 ✔ Container for-tests-golink

## 2. REST API

### 2.1. Preapare

In [42]:
import urllib3, json

def create_link_rest(url):
    data = json.dumps({
        "url": url,
    })
    r = urllib3.request(
        'POST',
        'http://localhost:8080/new',
        headers = {
            'Content-Type': 'application/json'
        },
        body = data
    )
    return r

def get_link(alias):
    r = urllib3.request(
        'GET',
        'http://localhost:8080/'+alias,
    )
    return r

### 2.2 Run tests

#### 2.2.1 Launch app

In [40]:
!sudo docker compose -f with_flags-compose.yml up -d

[+] Running 0/0
 ⠋ Network for-tests_default  Creating                                     0.1s 
[+] Running 1/1
 ✔ Network for-tests_default        Created                                0.2s 
 ⠋ Container for-tests-golinkcut-1  C...                                   0.0s 
 ⠋ Container golinkcut_db_test      Creat...                               0.0s 
[+] Running 3/3
 ✔ Network for-tests_default        Created                                0.2s 
 ✔ Container for-tests-golinkcut-1  C...                                   0.1s 
 ✔ Container golinkcut_db_test      Creat...                               0.1s 
[+] Running 3/3
 ✔ Network for-tests_default        Created                                0.2s 
 ✔ Container for-tests-golinkcut-1  C...                                   0.1s 
 ✔ Container golinkcut_db_test      Creat...                               0.1s 
[+] Running 3/3
 ✔ Network for-tests_default        Created                                0.2s 
 ✔ Container for-tests-golink

#### 2.2.2 Run tests

In [43]:
create_link_rest(gen_url())

{'shortLink': 'localhost/aMTpYzGLAC'}

#### 2.2.3 Stop app

In [31]:
!sudo docker compose -f with_flags-compose.yml down

[+] Running 0/0
 ⠋ Container for-tests-golinkcut-1  S...                                   0.1s 
 ⠋ Container golinkcut_db_test      Stopp...                               0.1s 
[+] Running 0/2
 ⠙ Container for-tests-golinkcut-1  S...                                   0.2s 
 ⠙ Container golinkcut_db_test      Stopp...                               0.2s 
[+] Running 1/2
 ✔ Container for-tests-golinkcut-1  R...                                   0.3s 
 ⠹ Container golinkcut_db_test      Stopp...                               0.3s 
[+] Running 1/2
 ✔ Container for-tests-golinkcut-1  R...                                   0.3s 
 ⠸ Container golinkcut_db_test      Stopp...                               0.4s 
[+] Running 2/2
 ✔ Container for-tests-golinkcut-1  R...                                   0.3s 
 ✔ Container golinkcut_db_test      Remov...                               0.5s 
 ⠋ Network for-tests_default        Removin...                             0.0s 
[+] Running 2/3
 ✔ Container 

## 3. Both